In [1]:
'''
Deep Learning Stock Price Predictor 
Author: Gil Akos
Objective: Create Benchmark Model with scikit learn regression
'''

'\nDeep Learning Stock Price Predictor \nAuthor: Gil Akos\nObjective: Create Benchmark Model with scikit learn regression\n'

In [2]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

#Import required libraries
import numpy as np
import pandas as pd
import yahoo_finance as yf
from yahoo_finance import Share
from pprint import pprint
import matplotlib.pyplot as plt

In [19]:
#Set Benchmark Dates
date_start, date_end = '2009-01-01', '2009-07-01'#'2015-06-18'

#Set Model Baseline
base_ticker = 'SPY'

print(pd.date_range(date_start,date_end))

DatetimeIndex(['2009-01-01', '2009-01-02', '2009-01-03', '2009-01-04',
               '2009-01-05', '2009-01-06', '2009-01-07', '2009-01-08',
               '2009-01-09', '2009-01-10',
               ...
               '2009-06-22', '2009-06-23', '2009-06-24', '2009-06-25',
               '2009-06-26', '2009-06-27', '2009-06-28', '2009-06-29',
               '2009-06-30', '2009-07-01'],
              dtype='datetime64[ns]', length=182, freq='D')


In [20]:
#Create a dataframe for stock data
stock_df= pd.DataFrame()#index = pd.date_range(date_start,date_end))
#Set the index to the date range

#Request STock Data
stock_data = Share(base_ticker)
stock_historical = stock_data.get_historical(date_start,date_end)

#Loop through the stock data
for d in stock_historical:
    stock_close.append(d['Adj_Close'])

stock_df['Adj_Close'] = stock_close

    
print(stock_df)

     Adj_Close
0    79.209531
1    78.883526
2    79.526948
3    78.789157
4    78.995057
5    77.313582
6    76.652998
7    76.592946
8     78.96074
9    78.670772
10   78.099212
11   78.175986
12   79.250865
13   81.110574
14   80.888772
15   80.530481
16   80.735218
17   80.325744
18   80.658444
19   80.641379
20   79.890673
21   80.914363
22   80.846115
23   78.935224
24    77.56177
25   76.495423
26   77.885943
27   75.940922
28   76.103009
29   77.212011
..         ...
845  70.098931
846  70.861245
847   70.81042
848  70.395384
849  73.774971
850  73.673333
851  71.632025
852  70.581729
853  70.929001
854  69.946468
855  70.158221
856  71.615088
857  74.020606
858  71.598144
859  70.878182
860  70.395384
861  70.090459
862   71.19158
863  68.243967
864   72.04706
865  71.488034
866  71.462625
867  73.783443
868  73.647917
869  75.460528
870  77.112211
871  76.798813
872  79.170456
873  78.645305
874  78.738477

[875 rows x 1 columns]
